У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv` [скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [ ]:
import pandas as pd

# Завантаження датасету
file_path = 'customer_segmentation_train.csv'  # Вкажіть правильний шлях до файлу
df = pd.read_csv(file_path)

# Попередній перегляд даних
df.head()


,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [ ]:
df.isna().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [ ]:
df.ID.nunique() # Перевірка, що колонка айді всюди унікальна

8068

In [ ]:
df.drop(columns=['ID'], inplace=True)  # Видаляємо айдішку, вона нам ні до чого

In [ ]:
# Обробка пропущених значень
df = df.dropna()  # Видалення рядків з пропущеними значеннями, ви можете зробити щось цікавіше))

In [ ]:
# Дослідженя категоріальних даних
for col in df.select_dtypes(include=['object']).columns:
  display(df[col].value_counts())

Gender
Male      3677
Female    2988
Name: count, dtype: int64

Ever_Married
Yes    3944
No     2721
Name: count, dtype: int64

Graduated
Yes    4249
No     2416
Name: count, dtype: int64

Profession
Artist           2192
Healthcare       1077
Entertainment     809
Doctor            592
Engineer          582
Executive         505
Lawyer            500
Marketing         233
Homemaker         175
Name: count, dtype: int64

Spending_Score
Low        3999
Average    1662
High       1004
Name: count, dtype: int64

Var_1
Cat_6    4476
Cat_4     849
Cat_3     634
Cat_2     362
Cat_7     166
Cat_1     104
Cat_5      74
Name: count, dtype: int64

Segmentation
D    1757
C    1720
A    1616
B    1572
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

target = 'Segmentation'
X = df.drop(columns=[target])
y = df[target]

# Розділення датасету на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.index

Index([ 463, 6977, 5545,  144, 3491,  507, 6235, 6328, 6349, 2438,
       ...
       6438,  563, 7595, 6919, 3735, 4557, 6266, 6309, 6505, 1057],
      dtype='int64', length=5332)

In [ ]:
# Кодування категоріальних ознак з OneHotEncoding, Spending_Score - OrdinalEncoding

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


categorical_features = X_train.select_dtypes(include=['object']).columns
num_features = X_train.select_dtypes(exclude=['object']).columns

# Використання OneHotEncoder для кодування категоріальних ознак
ohe_cols = np.setdiff1d(categorical_features, ['Spending_Score'])
ohe_encoder = OneHotEncoder(sparse=False, drop='if_binary')
ohe_encoded_features_train = ohe_encoder.fit_transform(X_train[ohe_cols])
ohe_encoded_features_test = ohe_encoder.transform(X_test[ohe_cols])

# Перетворення закодованих ознак у DataFrame
ohe_encoded_train_df = pd.DataFrame(ohe_encoded_features_train, columns=ohe_encoder.get_feature_names_out(ohe_cols), index=X_train.index)
ohe_encoded_test_df = pd.DataFrame(ohe_encoded_features_test, columns=ohe_encoder.get_feature_names_out(ohe_cols), index=X_test.index)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
ord_encoder = OrdinalEncoder(categories=[['Low', 'Average', 'High']])
ord_encoded_train_df = pd.DataFrame(
    ord_encoder.fit_transform(X_train[['Spending_Score']]), columns=['Spending_Score_Encoded'], index=X_train.index
)
ord_encoded_test_df = pd.DataFrame(
    ord_encoder.transform(X_test[['Spending_Score']]), columns=['Spending_Score_Encoded'], index=X_test.index
)

In [ ]:
# Об'єднання з оригінальним DataFrame, виключаючи початкові категоріальні ознаки

work_features = num_features.tolist()
categorical_feature_processed = ohe_encoded_train_df.columns.tolist() + ['Spending_Score_Encoded']
work_features.extend(ohe_encoded_train_df.columns.tolist())

X_train = pd.concat([X_train, ohe_encoded_train_df, ord_encoded_train_df], axis=1)[work_features]
X_test = pd.concat([X_test, ohe_encoded_test_df, ord_encoded_test_df], axis=1)[work_features]

In [ ]:
X_train.shape, X_test.shape

((5332, 22), (1333, 22))

In [ ]:
X_train

,Age,Work_Experience,Family_Size,Ever_Married_Yes,Gender_Male,Graduated_Yes,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,...,Profession_Homemaker,Profession_Lawyer,Profession_Marketing,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
463,42,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6977,49,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5545,41,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
144,43,0.0,5.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3491,48,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4557,62,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6266,48,3.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6309,63,0.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6505,57,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [ ]:
y_train.value_counts()

Segmentation
D    1387
C    1374
A    1301
B    1270
Name: count, dtype: int64

In [ ]:
X_train

,Age,Work_Experience,Family_Size,Ever_Married_Yes,Gender_Male,Graduated_Yes,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,...,Profession_Homemaker,Profession_Lawyer,Profession_Marketing,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
463,42,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6977,49,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5545,41,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
144,43,0.0,5.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3491,48,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4557,62,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6266,48,3.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6309,63,0.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6505,57,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
cat_feature_indeces = [id for id, name in enumerate(X_train.columns) if name in categorical_feature_processed]
cat_feature_indeces

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [ ]:
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

# Застосування SMOTE
smote = SMOTENC(random_state=42, categorical_features=cat_feature_indeces)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Застосування SMOTE-Tomek
smotetomek = SMOTETomek(random_state=42)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(X_train, y_train)

In [ ]:
X_train.shape, X_train_smote.shape, X_train_smotetomek.shape

((5332, 22), (5548, 22), (4686, 22))

In [ ]:
X_train_smote

,Age,Work_Experience,Family_Size,Ever_Married_Yes,Gender_Male,Graduated_Yes,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,...,Profession_Homemaker,Profession_Lawyer,Profession_Marketing,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
0,42,1.000000,2.000000,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,49,0.000000,1.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,41,0.000000,2.000000,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,43,0.000000,5.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,48,0.000000,2.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5543,50,1.000000,3.000000,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5544,47,0.611653,4.000000,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5545,40,1.000000,2.000000,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5546,27,0.976938,2.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

def train_evaluate_ovr(X_train, y_train, X_test, y_test):
  # Навчання моделі з використанням стратегії One-vs-Rest
  ovr_model = OneVsRestClassifier(LogisticRegression(solver='liblinear'))
  ovr_model.fit(X_train, y_train)

  ovr_predictions_train = ovr_model.predict(X_train)
  ovr_predictions_test = ovr_model.predict(X_test)

  # Оцінка моделі з використанням стратегії One-vs-Rest
  print(
      'Train',
      classification_report(y_train, ovr_predictions_train),
      'Test',
      classification_report(y_test, ovr_predictions_test),
      sep='\n'
  )

In [ ]:
# One-vs-Rest на оригінальних даних
train_evaluate_ovr(X_train, y_train, X_test, y_test)

Train
              precision    recall  f1-score   support

           A       0.44      0.51      0.47      1301
           B       0.42      0.13      0.20      1270
           C       0.49      0.69      0.57      1374
           D       0.65      0.69      0.67      1387

    accuracy                           0.51      5332
   macro avg       0.50      0.51      0.48      5332
weighted avg       0.50      0.51      0.49      5332

Test
              precision    recall  f1-score   support

           A       0.43      0.48      0.45       315
           B       0.41      0.15      0.22       302
           C       0.48      0.67      0.56       346
           D       0.66      0.68      0.67       370

    accuracy                           0.51      1333
   macro avg       0.49      0.50      0.47      1333
weighted avg       0.50      0.51      0.49      1333



In [ ]:
# One-vs-Rest на SMOTE upsampled даних
train_evaluate_ovr(X_train_smote, y_train_smote, X_test, y_test)

Train
              precision    recall  f1-score   support

           A       0.44      0.53      0.48      1387
           B       0.43      0.18      0.25      1387
           C       0.49      0.66      0.56      1387
           D       0.65      0.67      0.66      1387

    accuracy                           0.51      5548
   macro avg       0.50      0.51      0.49      5548
weighted avg       0.50      0.51      0.49      5548

Test
              precision    recall  f1-score   support

           A       0.41      0.50      0.45       315
           B       0.38      0.16      0.22       302
           C       0.49      0.65      0.56       346
           D       0.66      0.65      0.66       370

    accuracy                           0.50      1333
   macro avg       0.48      0.49      0.47      1333
weighted avg       0.49      0.50      0.48      1333



In [ ]:
# One-vs-Rest на SMOTE-Tomek upsampled даних
train_evaluate_ovr(X_train_smotetomek, y_train_smotetomek, X_test, y_test)

Train
              precision    recall  f1-score   support

           A       0.46      0.55      0.50      1164
           B       0.43      0.17      0.24      1169
           C       0.50      0.69      0.58      1215
           D       0.68      0.70      0.69      1200

    accuracy                           0.53      4748
   macro avg       0.52      0.53      0.50      4748
weighted avg       0.52      0.53      0.50      4748

Test
              precision    recall  f1-score   support

           A       0.40      0.49      0.44       315
           B       0.37      0.15      0.22       302
           C       0.49      0.65      0.56       346
           D       0.66      0.64      0.65       370

    accuracy                           0.50      1333
   macro avg       0.48      0.48      0.47      1333
weighted avg       0.49      0.50      0.48      1333



Суттєвої різниці між моделями на різних даних немає, скоріш за все через те, що дисбаланс класів був незначним. Але наша справа тут - потренуватись проводити експерименти, що ми і зробили.